In [15]:
SYMBOL =           ['+', '-', '*', '/', '^', 'LN', 'UM', 'VAR', '(', ')']
INPUT_PRECEDENCE = [1,    1,   3,   3,   6,   6,    7,     9,   11,   0]
STACK_PRECEDENCE = [2,    2,   4,   4,   5,   5,    8,    10,    0,  None]
RANK = [-1, -1, -1, -1, -1, -1, 1, None, None]
INDEX_LN = 5
INDEX_UM = 6
INDEX_VARIABLE = 7
INDEX_OPEN_BRACKET = 8
INDEX_CLOSE_BRACKET = 9

In [16]:
#TODO - clean it like getType()
def getIndex (symbol):
    if (symbol == 'LN'):
        index = INDEX_LN
        return index
    elif (symbol == 'UM'):
        index = INDEX_UM
    elif (symbol.isalpha()):
        index = INDEX_VARIABLE
        return index

    try:
        val = float (symbol)
        index = INDEX_VARIABLE
    except ValueError:
        index = SYMBOL.index (symbol)

    return index

In [17]:
def InfixToReversePolish (INFIX):
    #initialize
    REVERSEPOLISH = []
    STACK = []
    #append ')' to infix
    INFIX.append (SYMBOL[INDEX_CLOSE_BRACKET]) #= INFIX + ')'
    #push '(' on to the stack
    STACK.append (SYMBOL[INDEX_OPEN_BRACKET])
    for i in range(0, len(INFIX)):
        #read the next char in the infix
        NEXT = INFIX[i]
        #what is the index of next in the precedence and rank tables?
        index = getIndex (NEXT)
        if (len (STACK) == 0):
            print ('Invalid input string')
            return ['None']
        #if we encounter ')', we pop the stack till we find '('. we discard both '(' and ')'
        if index == INDEX_CLOSE_BRACKET:
            ch = STACK.pop()
            while getIndex (ch) != INDEX_OPEN_BRACKET:
                REVERSEPOLISH.append (ch)
                ch = STACK.pop()
            continue
        #while next input precedence is less than or equal to the top stack precedence    
        while (INPUT_PRECEDENCE[index] <= STACK_PRECEDENCE[getIndex(STACK[len(STACK) - 1])]):
            REVERSEPOLISH.append (STACK.pop())
        #push next on to the stack
        STACK.append (NEXT)
    #remove the ')' that was appended to INFIX
    INFIX.pop ()
    return REVERSEPOLISH

#print (InfixToReversePolish (INFIX))
#ex = ''.join (InfixToReversePolish (INFIX))
#print ('Reverse Polish Expression is', ex)

In [18]:
TYPE = {
    'CONSTANT': 0,
    'VARIABLE': 1,
    '+': 2,
    '-': 3,
    '*': 4,
    '/': 5,
    '^': 6,
    'LN': 7,
    'UM': 8
}

In [19]:
def getType (symbol):
    #operators
    if (symbol in TYPE):
        return TYPE[symbol]
    
    #variable
    if (symbol.isalpha()):
        return TYPE['VARIABLE']
    
    #real number constant
    try:
        val = float (symbol)
        return TYPE['CONSTANT']
    except ValueError:
        return None
class Node:
    def __init__ (self, symbol):
        self.LPTR = None
        self.RPTR = None
        self.SYMBOL = symbol
        self.TYPE = getType (symbol)
def MakeNode (symbol):
    return Node (symbol)
def MakeCopyOfNode (node):
    #do not copy LPTR, RPTR, 
    #otherwise the nodes of tree created by converting reverse polish to binary tree
    #will be pointed to by the tree representing its differentiation
    return Node (node.SYMBOL)

In [20]:
def UnaryNode (n1, n2):
    n1.RPTR = n2
    return n1
def BinaryNode (n1, n2, n3):
    n1.LPTR = n2
    n1.RPTR = n3
    return n1

In [21]:
def ReversePolishToBinaryTree (REVERSEPOLISH):
    STACK = []
    #from IPython.core.debugger import set_trace
    #set_trace()
    for symbol in REVERSEPOLISH:
        #create a node for the symbol
        node = MakeNode (symbol)
        #if node is a constant or a variable, push it on the stack
        if node.TYPE == TYPE['CONSTANT'] or node.TYPE == TYPE['VARIABLE']:
            STACK.append (node)
        #if node is an operator
        else:
            #if node is a unary operator
            if node.TYPE == TYPE['LN'] or node.TYPE == TYPE['UM']:
                #pop one operand from stack
                n2 = STACK.pop ()
                # create a unary operator node,
                node = UnaryNode (node, n2)
                # and push it back on the stack
                STACK.append (node)
            #if node is a binary operator
            else:
                #pop two operands from stack
                n3 = STACK.pop ()
                n2 = STACK.pop ()
                # create a binary operator node,
                node = BinaryNode (node, n2, n3)
                # and push it back on the stack
                STACK.append (node)
    #the last symbol is the root of the binary tree
    return node


#print (root.LPTR.RPTR.LPTR.RPTR.SYMBOL)

In [22]:
def printTree (root):
    if root == None:
        return
    printTree (root.LPTR)
    print (root.SYMBOL, end = " ")
    printTree (root.RPTR)

In [23]:
import math
def calculate (op, n1, n2):
    
    val = 0.0
    
    try:
        if (op == TYPE['+']):
            val = float(n1.SYMBOL) + float(n2.SYMBOL)
        elif (op == TYPE['-']):
            val = float(n1.SYMBOL) - float(n2.SYMBOL)
        elif (op == TYPE['*']):
            val = float(n1.SYMBOL) * float(n2.SYMBOL)
        elif (op == TYPE['/']):
            val = float(n1.SYMBOL) / float(n2.SYMBOL)
        elif (op == TYPE['^']):
            val = math.pow (float(n1.SYMBOL), float(n2.SYMBOL))
    except ValueError as error:
        raise error
        
    return str(val)

#print (calculate (TYPE['^'], Node('2'), Node('3')))

def purgeTree (root):
    if root == None:
        return

    purgeTree (root.LPTR)
    purgeTree (root.RPTR)

    if (root.RPTR != None): #unary or binary operator
        if(root.LPTR != None): #binary operator
            try:
                val = calculate (root.TYPE, root.LPTR, root.RPTR)
                root.TYPE = TYPE['CONSTANT']
                root.SYMBOL = val
                #purge the children
                root.LPTR = None
                root.RPTR = None
            except ValueError:
                return

#root = Node('^')
#root.LPTR = Node('2')
#root.RPTR = Node('3')
#purgeTree (root)
#print (root.SYMBOL)

In [24]:
def Differentiation (root, var): #var is the 'x' in differentiating w.r.t x, a.k.a d/dx

    if (root == None):
        return None
    
    #get the type of the root
    #depending on the type, apply the appropriate differentiation formula
    #recursively calling Differentiation mimics the chain rule

    #constants and variables
    
    type = root.TYPE
    if (type == TYPE['CONSTANT']):
        return MakeNode('0') #D(a number) = 0
    elif (type == TYPE['VARIABLE']):
        if root.SYMBOL == var:
            return MakeNode('1') #D(x) = 1
        else:
            return MakeNode('0') #D(a) = 0
    
    #now for the operators
    
    #get the differentiated left and right operands of the root
    rightOperand = Differentiation(root.RPTR, var)
    #if root operator is binary, get the left operand too
    if (root.LPTR != None):
        leftOperand = Differentiation(root.LPTR, var)
    else:
        leftOperand = None
    
    if (type == TYPE['+']): #D(u + v) = D(u) + D(v)
        return BinaryNode (MakeNode('+'), leftOperand, rightOperand)
    elif (type == TYPE['-']): #D(u - v) = D(u) - D(v)
        return BinaryNode (MakeNode('-'), leftOperand, rightOperand)
    elif (type == TYPE['*']): #D(u * v) = D(u)*v + D(v)*u
        return BinaryNode (MakeNode('+'), \
                           BinaryNode (MakeNode('*'), leftOperand, MakeCopyOfNode(root.RPTR)), \
                           BinaryNode (MakeNode('*'), rightOperand, MakeCopyOfNode(root.LPTR)) \
                          )
    elif (type == TYPE['/']): #D(u / v) = (D(u)/v) - (u*D(v)/v^2)
        DuBYv = BinaryNode (MakeNode('/'), leftOperand, MakeCopyOfNode(root.RPTR))
        uDv =   BinaryNode (MakeNode('*'), MakeCopyOfNode(root.LPTR), rightOperand)
        vSquare = BinaryNode (MakeNode('^'), MakeCopyOfNode(root.RPTR), MakeNode('2'))
        uDvBYvSquare = BinaryNode (MakeNode('/'), uDv, vSquare)
        return BinaryNode (MakeNode('-'), DuBYv, uDvBYvSquare)
    elif (type == TYPE['LN']):
        return BinaryNode (MakeNode('/'), rightOperand, MakeCopyOfNode(root.RPTR))
    elif (type == TYPE['UM']):
        return UnaryNode (MakeNode('-'), rightOperand)
    else:
        return None

In [31]:
#Keep a space between each token of the input
INFIX2 = '2 * x - a + x - x' #derivative is 0 * x + 2.0 - 0 + 1 - 1 
#INFIX2 = 'UM x + LN x + 2 / 3' #derivative is - 1 + 1 / x + 0.0 
print ('Input as string is', INFIX2)
INFIX = INFIX2.split()
#POLISH_TEST = 'abcd^^+efd/+*'

REVERSEPOLISH = InfixToReversePolish (INFIX)
print ('Input in Reverse Polish Notation is', REVERSEPOLISH)
#print (REVERSEPOLISH)
root = ReversePolishToBinaryTree (REVERSEPOLISH)
print ('Binary Tree as input for differentiation is', end = " ")
printTree (root)
print ("")

diffroot = Differentiation (root, 'x')
print ('Differentiation of input is', end = " ")
printTree (diffroot)
purgeTree (diffroot)
print ("")
print ('Cleaner version of Differentiation of input is', end = " ")
printTree (diffroot)

Input as string is 2 * x - a + x - x
Input in Reverse Polish Notation is ['2', 'x', '*', 'a', '-', 'x', '+', 'x', '-']
Binary Tree as input for differentiation is 2 * x - a + x - x 
Differentiation of input is 0 * x + 1 * 2 - 0 + 1 - 1 
Cleaner version of Differentiation of input is 0 * x + 2.0 - 0 + 1 - 1 